In [ ]:
cd CIFAR-10/AlexNet

In [4]:
import numpy as np, joblib
import sys, os, random
import matplotlib.pyplot as plt
import pickle, gzip
from tqdm import tqdm,tqdm_notebook
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [5]:
batch_size = 128
seed_num = 1001

# For reproducibility when you run the file with .py
torch.cuda.is_available()
torch.manual_seed(seed_num)
torch.cuda.manual_seed(seed_num)
np.random.seed(seed_num)
random.seed(seed_num)
torch.backends.cudnn.benchmark = True

torch.backends.cudnn.deterministic =True

In [6]:
# Normalization
train_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 
test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Splitting the training and test datasets
train_data = datasets.CIFAR10(os.getcwd(), train=True,
                              download=True, transform=train_transform)
test_data = datasets.CIFAR10(os.getcwd(), train=False,
                             download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Split the training set indices into training and validation set indices using 80:20 ratio
np.random.seed(seed_num)
len_trainset = len(train_data)
index_list = list(range(len_trainset))
np.random.shuffle(index_list)
split_index = 40000
train_indices, valid_indices =  index_list[:split_index], index_list[split_index:]

# Creating Samplers for training and validation set using the indices
np.random.seed(seed_num)
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

torch.manual_seed(seed_num)

train_iterator = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
val_iterator = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_iterator = DataLoader(test_data, batch_size=batch_size, shuffle=True)


In [8]:
# AlexNet Model

class AlexNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1),     # in_channels, out_channels, kernel_size, stride, padding
            nn.BatchNorm2d(32),
            nn.ReLU(inplace = True),
            nn.Conv2d(32, 64, 3, padding = 1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace = True),

            nn.Conv2d(64, 128, 3, padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace = True),
            nn.Conv2d(128, 256, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.Conv2d(256, 256, 3, padding = 1),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(256 * 7 * 7, 256),
            nn.ReLU(inplace = True),
            nn.Linear(256,256),
            nn.ReLU(inplace = True),
            nn.Linear(256, output_dim),
        )


    def forward(self, x):
        x = self.features(x)
        h = x.view(x.size(0), -1)
        x = self.classifier(h)
        
        return x

In [9]:
torch.manual_seed(seed_num)
unit=128

# Summary
model = AlexNet(10)
print("Model:\n",model)

Model:
 AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): Linear(in_features=1254

In [10]:
# Layer names
layer_name = [n for n, p in model.named_parameters()]
print(layer_name)

['features.0.weight', 'features.0.bias', 'features.1.weight', 'features.1.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.7.weight', 'features.7.bias', 'features.9.weight', 'features.9.bias', 'features.11.weight', 'features.11.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.5.weight', 'classifier.5.bias']


In [11]:
# Loading the weights of ternary model 
model = torch.load("AlexNet_cifar10_Quant.pt")
model = model.cuda()
print("Loading weights done !")

# Total number of ternary weights (+w, -w)
totalParams = 0
for i in layer_name:
  print(i,(model.state_dict()[i] !=0).sum())
  totalParams +=  (model.state_dict()[i] !=0).sum()
    
print("Total Parameters:",totalParams, '\n')

Loading weights done !
features.0.weight tensor(528, device='cuda:0')
features.0.bias tensor(29, device='cuda:0')
features.1.weight tensor(32, device='cuda:0')
features.1.bias tensor(13, device='cuda:0')
features.3.weight tensor(683, device='cuda:0')
features.3.bias tensor(37, device='cuda:0')
features.6.weight tensor(467, device='cuda:0')
features.6.bias tensor(115, device='cuda:0')
features.7.weight tensor(64, device='cuda:0')
features.7.bias tensor(48, device='cuda:0')
features.9.weight tensor(1001, device='cuda:0')
features.9.bias tensor(52, device='cuda:0')
features.11.weight tensor(1510, device='cuda:0')
features.11.bias tensor(143, device='cuda:0')
classifier.1.weight tensor(8254, device='cuda:0')
classifier.1.bias tensor(96, device='cuda:0')
classifier.3.weight tensor(839, device='cuda:0')
classifier.3.bias tensor(142, device='cuda:0')
classifier.5.weight tensor(948, device='cuda:0')
classifier.5.bias tensor(8, device='cuda:0')
Total Parameters: tensor(15009, device='cuda:0') 


In [12]:
# Model's performance on test set

correct_count, all_count = 0, 0
model.eval()
for images,labels in test_iterator:
      for image,label in zip(images,labels):
        if torch.cuda.is_available():
            img = image.cuda()
            lab = label.cuda()
            img = img[None,].type('torch.cuda.FloatTensor')

        with torch.no_grad():
            output_ = model(img) 

        pred_label = output_.argmax()

        if(pred_label.item()==lab.item()):
          correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Test Accuracy =", (correct_count/(all_count)))


Number Of Images Tested = 10000

Model Test Accuracy = 0.5374


In [13]:
# For each layer, model's ternary weights
state_dict = model.state_dict()

layer_distinct_weights = {}

for i in layer_name:
  imd = torch.unique(model.state_dict()[i])
  print(i+ ' hidden layer dimension', model.state_dict()[i].shape)
  print("Unique values of weight in "+ i+ " th hidden layer : ", imd)
  layer_distinct_weights[i] = imd.cpu().numpy().tolist()
  print()


features.0.weight hidden layer dimension torch.Size([32, 3, 3, 3])
Unique values of weight in features.0.weight th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.0.bias hidden layer dimension torch.Size([32])
Unique values of weight in features.0.bias th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.1.weight hidden layer dimension torch.Size([32])
Unique values of weight in features.1.weight th hidden layer :  tensor([1.], device='cuda:0')

features.1.bias hidden layer dimension torch.Size([32])
Unique values of weight in features.1.bias th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.3.weight hidden layer dimension torch.Size([64, 32, 3, 3])
Unique values of weight in features.3.weight th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.3.bias hidden layer dimension torch.Size([64])
Unique values of weight in features.3.bias th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

features.6.weight hi

In [14]:
img_0 = []; var_0 = 0
img_1 = []; var_1 = 0
img_2 = []; var_2 = 0
img_3 = []; var_3 = 0
img_4 = []; var_4 = 0
img_5 = []; var_5 = 0
img_6 = []; var_6 = 0
img_7 = []; var_7 = 0
img_8 = []; var_8 = 0
img_9 = []; var_9 = 0

In [15]:
correct_count, all_count = 0, 0
np.random.seed(seed_num)

num = 1000

model.eval()
for images,labels in train_iterator:
      for image,label in zip(images,labels):

        if label == 0:
          if var_0 == num:
            continue
          img_0.append(image.numpy())
          var_0 +=1

        elif label == 1:
          if var_1 == num:
            continue
          img_1.append(image.numpy())
          var_1 +=1

        elif label == 2:
          if var_2 == num:
            continue
          img_2.append(image.numpy())
          var_2 +=1

        elif label == 3:
          if var_3 == num:
            continue
          img_3.append(image.numpy())
          var_3 +=1

        elif label == 4:
          if var_4 == num:
            continue
          img_4.append(image.numpy())
          var_4 +=1

        elif label == 5:
          if var_5 == num:
            continue
          img_5.append(image.numpy())
          var_5 +=1

        elif label == 6:
          if var_6 == num:
            continue
          img_6.append(image.numpy())
          var_6 +=1

        elif label == 7:
          if var_7 == num:
            continue
          img_7.append(image.numpy())
          var_7 +=1

        elif label == 8:
          if var_8 == num:
            continue
          img_8.append(image.numpy())
          var_8 +=1

        elif label == 9:
          if var_9 == num:
            continue
          img_9.append(image.numpy())
          var_9 +=1

        if torch.cuda.is_available():
            img = image.cuda()
            lab = label.cuda()
            img = img[None,].type('torch.cuda.FloatTensor')

        with torch.no_grad():
            output_ = model(img) 

        pred_label = output_.argmax()

        if(pred_label.item()==lab.item()):
          correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Train Accuracy =", (correct_count/(all_count)))

Number Of Images Tested = 10000

Model Train Accuracy = 0.5541


In [16]:
corr_images_ = [img_0, img_1, img_2, img_3, img_4, img_5, img_6, img_7, img_8, img_9]
corr_images = np.vstack(corr_images_)
corr_images.shape

(10000, 3, 32, 32)

In [17]:
corr_images = torch.tensor(corr_images).type(torch.FloatTensor).to('cuda')

corr_label = []
check_label = []

model.eval()
for img in corr_images:
  img = img[None,].type('torch.cuda.FloatTensor')
  with torch.no_grad():
    pred = model(img).argmax()
  corr_label.append(pred)
  check_label.append(pred.item())

digit,count = np.unique(check_label, return_counts=True)

# Model's prediction on pseudorandom images
print("\nUnique digits : ",digit)

print("\n counts : ",count)


Unique digits :  [0 1 2 3 4 5 6 7 8 9]

 counts :  [ 711  873  955 1133  879  852 1014 1173 1564  846]


In [18]:
pos_neg_layerwise = {}

for i in layer_name:
  b = layer_distinct_weights[i]

  if len(b) == 3:
    neg, zero, pos = b
    pos_neg_layerwise[i] = {}
    pos_neg_layerwise[i]['pos'] = pos
    pos_neg_layerwise[i]['neg'] = neg
    
  elif len(b) == 2:
    un1, un2 = b

    if un2 == 0:
      neg, zero = b
      pos_neg_layerwise[i] = {}
      pos_neg_layerwise[i]['neg'] = neg
      
    elif un1 == 0:
      zero, pos = b
      pos_neg_layerwise[i] = {}
      pos_neg_layerwise[i]['pos'] = pos

    else:
      neg, pos = b
      pos_neg_layerwise[i] = {}
      pos_neg_layerwise[i]['pos'] = pos
      pos_neg_layerwise[i]['neg'] = neg

  else:
     un = b[0] 

     if un > 0:
       pos_neg_layerwise[i] = {}
       pos_neg_layerwise[i]['pos'] = un

     elif un < 0:
       pos_neg_layerwise[i] = {}
       pos_neg_layerwise[i]['neg'] = un

     else:
       pos_neg_layerwise[i] = {}
       pos_neg_layerwise[i]['zero'] = 0


In [19]:
layer_distinct_weights

{'classifier.1.bias': [-1.0, 0.0, 1.0],
 'classifier.1.weight': [-1.0, 0.0, 1.0],
 'classifier.3.bias': [-1.0, 0.0, 1.0],
 'classifier.3.weight': [-1.0, 0.0, 1.0],
 'classifier.5.bias': [-1.0, 0.0, 1.0],
 'classifier.5.weight': [-1.0, 0.0, 1.0],
 'features.0.bias': [-1.0, 0.0, 1.0],
 'features.0.weight': [-1.0, 0.0, 1.0],
 'features.1.bias': [-1.0, 0.0, 1.0],
 'features.1.weight': [1.0],
 'features.11.bias': [-1.0, 0.0, 1.0],
 'features.11.weight': [-1.0, 0.0, 1.0],
 'features.3.bias': [-1.0, 0.0, 1.0],
 'features.3.weight': [-1.0, 0.0, 1.0],
 'features.6.bias': [-1.0, 0.0, 1.0],
 'features.6.weight': [-1.0, 0.0, 1.0],
 'features.7.bias': [-1.0, 0.0, 1.0],
 'features.7.weight': [0.0, 1.0],
 'features.9.bias': [-1.0, 0.0, 1.0],
 'features.9.weight': [-1.0, 0.0, 1.0]}

In [20]:
class PseudoData(Dataset):
    
    def __init__(self, data, label):
        
        self.data = data
        self.label = label

    def __getitem__(self, index):
        
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)

corr_label = torch.tensor(corr_label).type(torch.FloatTensor).to('cuda')
pseudo_dataLoader = DataLoader(dataset = PseudoData(corr_images, corr_label), batch_size = 1000, shuffle=False) 

In [21]:
def getResult(test_example, test_label, model, st):
    model.load_state_dict(st)
    model.eval()
    with torch.no_grad():
        output_ = model(test_example.float()) 
        pred = output_.data.max(1, keepdim=True)[1]
        z = pred.eq(test_label.data.view_as(pred)).flatten()
        return torch.where(z == False)[0]

In [22]:
def populateResults(idx, batch_id, old_state, new_state, w_images, n_ic, n_oc, row, column, weight_name):
  key = 'img_id'+str(idx)+'_batch_'+str(batch_id)
  if key not in w_images:
    w_images[key] = {}
    w_images[key]['location'] = []
    w_images[key]['location'].append((weight_name, n_ic, n_oc, row, column, old_state, new_state))
    w_images[key]['weight_states'] = []
    w_images[key]['weight_states'].append(weight_name + ' : ' + str(old_state) + ' --> ' + str(new_state))

  else:
    w_images[key]['location'].append((weight_name, n_ic, n_oc, row, column, old_state, new_state))
    w_images[key]['weight_states'].append(weight_name + ' : ' + str(old_state) + ' --> ' + str(new_state))

In [23]:
def populateResults1(idx, batch_id, old_state, new_state, w_images, row, column, weight_name):
  key = 'img_id'+str(idx)+'_batch_'+str(batch_id)
  if key not in w_images:
    w_images[key] = {}
    w_images[key]['location'] = []
    w_images[key]['location'].append((weight_name, row, column, old_state, new_state))
    w_images[key]['weight_states'] = []
    w_images[key]['weight_states'].append(weight_name + ' : ' + str(old_state) + ' --> ' + str(new_state))

  else:
    w_images[key]['location'].append((weight_name, row, column, old_state, new_state))
    w_images[key]['weight_states'].append(weight_name + ' : ' + str(old_state) + ' --> ' + str(new_state))

In [24]:
# For bias weight mutation

def bias_mutation(st, superSet, w_images, img,label, batch_id, weight_name):

  if ('pos' in pos_neg_layerwise[weight_name]) and ('neg' in pos_neg_layerwise[weight_name]):
    un = pos_neg_layerwise[weight_name]['pos'], pos_neg_layerwise[weight_name]['neg']

    var_neg1 = torch.where(st==un[1])[0]
    var_1 = torch.where(st==un[0])[0]

  else:
    try:
      un = pos_neg_layerwise[weight_name]['pos']

      var_neg1 = torch.where(st==un*1000)[0]
      var_1 = torch.where(st==un)[0]

    except KeyError:
      un = pos_neg_layerwise[weight_name]['neg']

      var_neg1 = torch.where(st==un)[0]
      var_1 = torch.where(st==un*1000)[0]


  if (var_neg1.nelement() == 0.) and (var_1.nelement() == 0.):
    pass
  else:
    for column in torch.cat((var_neg1, var_1), 0):
      row = 0
      column = column.item()

      org_val_real = st[column].item() # For bias part

      if type(un) == tuple:
          pos, neg = un   # Positive, Negative 
          if (org_val_real < 0.) and (org_val_real == neg):
            org_val = neg

            old_state = neg
            new_state = 0.
            superSet.add((weight_name, row, column, -1, 0))

            st[column] = new_state

            """getResult() will tell whether the passed test example is classified to other class
            or not after mutation, if it is classified to other class, then it returns True else False and 
            populateResults() will maintain the record of test examples which has been misclassified
            like location(row(which neuron), column(indices)), old_state(old_value of weight element), 
            new_state (mutated value of weight element), images"""

            
            [populateResults1(idx.item(), batch_id, -1, 0, w_images, row, column, weight_name) \
             for idx in getResult(img, label, model, state_dict)]
          
            new_state = pos
            superSet.add((weight_name, row, column, -1, 1))

            st[column] = new_state

            [populateResults1(idx.item(), batch_id, -1, 1, w_images, row, column, weight_name) \
             for idx in getResult(img, label, model, state_dict)]
            
            st[column] = org_val

          elif (org_val_real > 0.) and (org_val_real == pos):

            org_val = pos
            old_state = pos
            new_state = 0.
            superSet.add((weight_name, row, column, 1, 0))
            st[column] = new_state

            [populateResults1(idx.item(), batch_id, 1, 0, w_images, row, column, weight_name) \
             for idx in getResult(img, label, model, state_dict)]
          
            new_state = neg
            superSet.add((weight_name, row, column, 1, -1))

            st[column] = new_state
            
            [populateResults1(idx.item(), batch_id, 1, -1, w_images, row, column, weight_name) \
             for idx in getResult(img, label, model, state_dict)]
            
            st[column] = org_val


      
      ############# Negative ################################
      elif (un < 0) and (org_val_real == un):
          org_val = un
          old_state = un
          new_state = 0.
          superSet.add((weight_name, row, column, -1, 0))
          st[column] = new_state
            
          [populateResults1(idx.item(), batch_id, -1, 0, w_images, row, column, weight_name) \
           for idx in getResult(img, label, model, state_dict)]
          
          st[column] = org_val


      elif (un > 0) and (org_val_real == un):  ############# Positive  ################################
          org_val = un
          old_state = un
          new_state = 0.
          superSet.add((weight_name, row, column, 1, 0))
          st[column] = new_state
          [populateResults1(idx.item(), batch_id, 1, 0, w_images, row, column, weight_name) \
           for idx in getResult(img, label, model, state_dict)]

          st[column] = org_val


      #################################################################################################################

In [25]:
# Saving length of shape for each layer
layers_shape = {}

for i in layer_name:
  layers_shape[i] = len(state_dict[i].shape)

In [26]:
print(layers_shape)

{'features.0.weight': 4, 'features.0.bias': 1, 'features.1.weight': 1, 'features.1.bias': 1, 'features.3.weight': 4, 'features.3.bias': 1, 'features.6.weight': 4, 'features.6.bias': 1, 'features.7.weight': 1, 'features.7.bias': 1, 'features.9.weight': 4, 'features.9.bias': 1, 'features.11.weight': 4, 'features.11.bias': 1, 'classifier.1.weight': 2, 'classifier.1.bias': 1, 'classifier.3.weight': 2, 'classifier.3.bias': 1, 'classifier.5.weight': 2, 'classifier.5.bias': 1}


In [27]:
# Injecting fault by mutating the ternary weight and check whether it is classified to other class than before by the model. 

fetchimg_totImg_coverage_totCov = []

w_images = {}
superSet = set()


batch_id = 0
for img,label  in tqdm(pseudo_dataLoader):
  

    for weight_name in layer_name:

        # Selecting hidden layer to iterate its weight elements
        st = state_dict[weight_name]


        if ('bias' in weight_name) or (layers_shape[weight_name] == 1):
          bias_mutation(st, superSet, w_images, img,label, batch_id, weight_name)
          continue

        if ('conv' in weight_name) or (layers_shape[weight_name] == 4):

          ic, oc, k,k1 = st.shape # ic = in_channel, oc= out_channel, k and k1 is the number of rows and columns of a kernel

          for n_ic in range(ic):
            for n_oc in range(oc):
              kt =  st[n_ic][n_oc] # Fetching Kernel matrix or tensor

              # Iterating number of neurons for the selected hidden layer
              for row in range(k):

                # Fetching indices for -w and w weight value for the selected kernel
                """Since each kernel has weight vector, so by fetching indices
                we will come to know, which indices of the kernel contains weights -w or w weight"""

                ###########################################################################################

                if ('pos' in pos_neg_layerwise[weight_name]) and ('neg' in pos_neg_layerwise[weight_name]):
                  un = pos_neg_layerwise[weight_name]['pos'], pos_neg_layerwise[weight_name]['neg']

                  var_neg1 = torch.where(kt[row]==un[1])[0]
                  var_1 = torch.where(kt[row]==un[0])[0]

                else:
                  try:
                    un = pos_neg_layerwise[weight_name]['pos']

                    var_neg1 = torch.where(kt[row]==un*1000)[0]
                    var_1 = torch.where(kt[row]==un)[0]

                  except KeyError:
                    un = pos_neg_layerwise[weight_name]['neg']

                    var_neg1 = torch.where(kt[row]==un)[0]
                    var_1 = torch.where(kt[row]==un*1000)[0]

                ##########################################################################################

                if (var_neg1.nelement() == 0.) and (var_1.nelement() == 0.):
                  continue
                else:
                  for column in torch.cat((var_neg1, var_1), 0):

                    column = column.item()

                    org_val_real = kt[row][column].item()

                    ##################################################################################################################

                    if type(un) == tuple:
                      pos, neg = un  # Positive, Negative

                      if (org_val_real < 0.) and (org_val_real == neg):
                        org_val = neg
                        old_state = neg
                        new_state = 0.
                        superSet.add((weight_name, n_ic, n_oc, row, column, -1, 0))

                        st[n_ic][n_oc][row][column] = new_state
                        
                        """getResult() will tell whether the passed test example is classified to other class
                        or not after mutation, if it is classified to other class, then it returns True else False and 
                        populateResults() will maintain the record of test examples which has been misclassified
                        like location(row(which neuron), column(indices)), old_state(old_value of weight element), 
                        new_state (mutated value of weight element), images"""

                        
                        [populateResults(idx.item(), batch_id, -1, 0, w_images, n_ic, n_oc, row, column, weight_name) \
                         for idx in getResult(img, label, model, state_dict)]
                        
                        new_state = pos
                        superSet.add((weight_name, n_ic, n_oc, row, column, -1, 1))
                        st[n_ic][n_oc][row][column] = new_state

                            
                        [populateResults(idx.item(), batch_id, -1, 1, w_images, n_ic, n_oc, row, column, weight_name) \
                         for idx in getResult(img, label, model, state_dict)]
                        st[n_ic][n_oc][row][column] = org_val # Setting back to original value

                    
                      elif (org_val_real > 0.) and (org_val_real == pos):
                        org_val = pos
                        old_state = pos
                        new_state = 0.
                        superSet.add((weight_name, n_ic, n_oc, row, column, 1, 0))

                        st[n_ic][n_oc][row][column] = new_state
                        
                        [populateResults(idx.item(), batch_id, 1, 0, w_images, n_ic, n_oc, row, column, weight_name) \
                         for idx in getResult(img, label, model, state_dict)]
                        
                        new_state = neg
                        superSet.add((weight_name, n_ic, n_oc, row, column, 1, -1))
                        st[n_ic][n_oc][row][column] = new_state

                            
                        [populateResults(idx.item(), batch_id, 1, -1, w_images, n_ic, n_oc, row, column, weight_name) \
                         for idx in getResult(img, label, model, state_dict)]
                        
                        st[n_ic][n_oc][row][column] = org_val # Setting back to original value




                    elif (un < 0) and (org_val_real == un): ########################## Negative ###########################################
                      org_val = un
                      old_state = un
                      new_state = 0.
                      superSet.add((weight_name,  n_ic, n_oc, row, column, -1, 0))
                      st[n_ic][n_oc][row][column] = new_state
                        
                      [populateResults(idx.item(), batch_id, -1, 0, w_images, n_ic, n_oc, row, column, weight_name) \
                       for idx in getResult(img, label, model, state_dict)]
                      
                      st[n_ic][n_oc][row][column] = org_val


                    elif (un > 0) and (org_val_real == un): ########################## Positive ###########################################
                      org_val = un
                      old_state = un
                      new_state = 0.
                      superSet.add((weight_name, n_ic, n_oc, row, column, 1, 0))
                      st[n_ic][n_oc][row][column] = new_state
                        
                      [populateResults(idx.item(), batch_id, 1, 0, w_images, n_ic, n_oc, row, column, weight_name) \
                       for idx in getResult(img, label, model, state_dict)]
                      st[n_ic][n_oc][row][column] = org_val


    
            
        
        # For fully connected layers
        elif layers_shape[weight_name] == 2:
          rows = state_dict[weight_name].shape[0]

          # Iterating number of neurons for the selected hidden layer
          for row in range(rows):

            # Fetching indices for -w and w weight value for the selected neuron
            """Since each neuron has weight vector, so by fetching indices
            we will come to know, which indices of the neuron contains weights -w or w weight"""

            #########################################################################################

            if ('pos' in pos_neg_layerwise[weight_name]) and ('neg' in pos_neg_layerwise[weight_name]):
              un = pos_neg_layerwise[weight_name]['pos'], pos_neg_layerwise[weight_name]['neg']

              var_neg1 = torch.where(st[row]==un[1])[0]
              var_1 = torch.where(st[row]==un[0])[0]

            else:
              try:
                un = pos_neg_layerwise[weight_name]['pos']

                var_neg1 = torch.where(st[row]==un*1000)[0]
                var_1 = torch.where(st[row]==un)[0]

              except KeyError:
                un = pos_neg_layerwise[weight_name]['neg']

                var_neg1 = torch.where(st[row]==un)[0]
                var_1 = torch.where(st[row]==un*1000)[0]


            #########################################################################################


            if (var_neg1.nelement() == 0.) and (var_1.nelement() == 0.):
              continue
            else:
              for column in torch.cat((var_neg1, var_1), 0):

                column = column.item()

                org_val_v1 = st[row][column].item()


                #########################################################################################

                if type(un) == tuple:
                  pos, neg = un  # Positive, Negative

                  if (org_val_v1 < 0.) and (neg == org_val_v1):

                    org_val = neg
                    old_state = neg
                    new_state = 0.
                    superSet.add((weight_name, row, column, -1, 0))

                    st[row][column] = new_state
                    
                    [populateResults1(idx.item(), batch_id, -1, 0, w_images, row, column, weight_name) \
                     for idx in getResult(img, label, model, state_dict)]
                    
                    new_state = pos
                    superSet.add((weight_name, row, column, -1, 1))
                    st[row][column] = new_state
                        
                    [populateResults1(idx.item(), batch_id, -1, 1, w_images, row, column, weight_name) \
                     for idx in getResult(img, label, model, state_dict)]
                    st[row][column] = org_val


                  elif (org_val_v1 > 0.) and (pos == org_val_v1):

                    org_val = pos
                    old_state = pos
                    new_state = 0.
                    superSet.add((weight_name, row, column, 1, 0))

                    st[row][column] = new_state
                    
                    [populateResults1(idx.item(), batch_id, 1, 0, w_images, row, column, weight_name) \
                     for idx in getResult(img, label, model, state_dict)]
                    
                    new_state = neg
                    superSet.add((weight_name, row, column, 1, -1))
                    st[row][column] = new_state
                        
                    [populateResults1(idx.item(), batch_id, 1, -1, w_images, row, column, weight_name) \
                     for idx in getResult(img, label, model, state_dict)]
                    st[row][column] = org_val


 
                elif (un < 0) and (un == org_val_v1):  ################# Negative ###############################################################
                  org_val = un  
                  old_state = un
                  new_state = 0.
                  superSet.add((weight_name, row, column, -1, 0))
                  st[row][column] = new_state
                    
                  [populateResults1(idx.item(), batch_id, -1, 0, w_images, row, column, weight_name) \
                   for idx in getResult(img, label, model, state_dict)]

                  st[row][column] = org_val


                elif (un > 0) and (un == org_val_v1):  ################# Positive ###############################################################
                  org_val = un
                  old_state = un
                  new_state = 0.
                  superSet.add((weight_name, row, column, 1, 0))
                  st[row][column] = new_state
                    
                  [populateResults1(idx.item(), batch_id, 1, 0, w_images, row, column, weight_name) \
                   for idx in getResult(img, label, model, state_dict)]
                  
                  st[row][column] = org_val


    batch_id += 1


    # Fetching locations and total number of images
    ################################################

    print(f"Total number of elements in superSet is : {len(superSet)}.",'\n')

    # joblib.dump(w_images, open('w_images_'+str(batch_id)+'.pkl','wb')) 

    image_locCounts = [] # (image_id,locationCounts)

    for img in w_images.keys():
        locCounts = len(w_images[img]['location'])
        image_locCounts.append((locCounts,img))
            
    image_locCounts.sort(reverse=True)


    netSetOfImages = []
    progressingSet = []
    max_locCounts_img = image_locCounts[0][1] # An image that has encompassed maximum locations.

    netSetOfImages.append(max_locCounts_img)
    progressingSet += set(w_images[max_locCounts_img]['location'])


    for i in range(1,len(image_locCounts)):
        img = image_locCounts[i][1]
        locs = w_images[img]['location']

        if set(locs).issubset(progressingSet):
            pass
        else:
            progressingSet += list(set(locs) - set(progressingSet))
            netSetOfImages.append(img)


    if set(superSet).issubset(progressingSet):
        print("All match done!")
    else:
        print("There are {} locations that did not match.".format(len(set(superSet) - set(progressingSet))))

    print()
    print("Total number of locations {} are encompassed out of {}.".format(len(set(progressingSet)), len(superSet)))


    print("Total number of elements in a weight matrix is ", len(superSet))

    print("Total number of images fetched ",len(netSetOfImages),'\n')

    fetchimg_totImg_coverage_totCov.append(( len(netSetOfImages), len(set(progressingSet)), len(superSet) ))

    # joblib.dump(fetchimg_totImg_coverage_totCov, open('res'+str(numImgs)+'.pkl','wb')) 

    # Superset 
    # joblib.dump(superSet, open('superSet.pkl','wb'))    

                
      
    print("Output \n")
    print(fetchimg_totImg_coverage_totCov)

    #joblib.dump(fetchimg_totImg_coverage_totCov, open('op_128.pkl', 'wb'))


  0%|          | 0/10 [00:00<?, ?it/s]

Total number of elements in superSet is : 29922. 



 10%|█         | 1/10 [36:36<5:29:24, 2196.09s/it]

There are 11293 locations that did not match.

Total number of locations 18629 are encompassed out of 29922.
Total number of elements in a weight matrix is  29922
Total number of images fetched  178 

Output 

[(178, 18629, 29922)]
Total number of elements in superSet is : 29922. 



 20%|██        | 2/10 [1:13:20<4:53:25, 2200.72s/it]

...

There are 2933 locations that did not match.

Total number of locations 26989 are encompassed out of 29922.
Total number of elements in a weight matrix is  29922
Total number of images fetched  428 

Output 

[(178, 18629, 29922), (247, 21716, 29922), (315, 24413, 29922), (363, 26078, 29922), (390, 26386, 29922), (401, 26640, 29922), (409, 26880, 29922), (428, 26989, 29922)]
Total number of elements in superSet is : 29922. 



 90%|█████████ | 9/10 [5:32:08<37:00, 2220.90s/it]  

There are 2763 locations that did not match.

Total number of locations 27159 are encompassed out of 29922.
Total number of elements in a weight matrix is  29922
Total number of images fetched  437 

Output 

[(178, 18629, 29922), (247, 21716, 29922), (315, 24413, 29922), (363, 26078, 29922), (390, 26386, 29922), (401, 26640, 29922), (409, 26880, 29922), (428, 26989, 29922), (437, 27159, 29922)]
Total number of elements in superSet is : 29922. 



100%|██████████| 10/10 [6:09:21<00:00, 2216.15s/it]

There are 2567 locations that did not match.

Total number of locations 27355 are encompassed out of 29922.
Total number of elements in a weight matrix is  29922
Total number of images fetched  437 

Output 

[(178, 18629, 29922), (247, 21716, 29922), (315, 24413, 29922), (363, 26078, 29922), (390, 26386, 29922), (401, 26640, 29922), (409, 26880, 29922), (428, 26989, 29922), (437, 27159, 29922), (437, 27355, 29922)]


In [28]:
# Fault coverage 
fetchimg_totImg_coverage_totCov[-1][1]/fetchimg_totImg_coverage_totCov[-1][2]

0.9142102800614932